# Week 3: Classification
#### Design Thinking and Predictive Pipelines for Data Products

(Same as Week 1) *An Introduction to Statistical Learning* introduces supervised learning as "building a statistical model for predicting, or estimating, an *output* based on one or more *inputs*". This is the core difference between supervised and unsupervised learning; in unsupervised learning, there is no direct output, but we can still learn about relationships and structures in the data. As a general rule of thumb, we tend to refer to problems with a *quantitative response* as regression problems, while those involving a *qualitative response* are often referred to as classification problems. The type(s) of predictors are not usually that important when making this distinction.

This week we were introduced to the concept of classification and several methods of doing this, such as nearest neighbors, logistic regression, and support vector machines. In this notebook, we will go over basic classification schemes as shown in lecture videos.

---

## Part 1: Logistic Regression

#### Documentation

* https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
* https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

#### Introduction

*"Logistic regression is used to describe data and to explain the relationship between one dependent **binary** variable and one or more nominal, ordinal, interval or ratio-level independent variables."* [-Statistic Solutions](https://www.statisticssolutions.com/what-is-logistic-regression/)

We use logistic regression when the output is **binary** and (often) **qualitative** (e.g. yes/no) to find the probability of the outcomes. For multiple categories (e.g. cat/dog/fish), we use multi-class logistic regression -- essentially, we re-run binary classification multiple times, once for each class. Some examples of questions answerable by logistic regression are "Do private tutoring hours, extracurriculars, and number of days of school attendance have an influence on the probability of a child graduating high school?" and "How does the probability of getting a heart attack (yes vs. no) change for every additional pound a person is overweight and for every pack of cigarettes smoked per day?"

In logistic regression, we cannot assume there is approximately a linear relationship between $x$ and $y$. Instead, we use the **logistic function** (below) to provide outputs between 0 and 1. The coefficients $\beta_0$ and $\beta_1$ are unknown, and must be estimated based on the available training data.

![Logistical function](https://imgur.com/IHCEI4C.png "Logistical function")

You may compare this to linear regression in the image below. The logistic function will always give us the S-shape on the right, and so we will always have a sensible prediction:

![Linear vs logistic regression graphs](https://imgur.com/QcFy56l.png "Linear vs logistic regression graphs")

### The Data

Take a look at the `business.json` file in the Week 2 folder. This dataset contains 10,000 businesses and their information from Yelp, with features such as the name of the business and location. For this notebook, we will be doing a simple logistic regression, looking at how well a business' overall rating and number of reviews can predict whether it is open. 

### Reading the Data
Specify the path of the file. You may need to change the given path according to your local environment. This should be familiar if you took Course 1 (*Basic Data Ingestion, Processing, and Visualization*) of the Python Data Products specialization already. We will be using `sklearn` to perform our classification.

In [ ]:
import urllib.request # read and open URLs
from sklearn import linear_model

In [ ]:
def parseData(filename):
  for line in urllib.request.urlopen(filename):
    yield eval(line)

# Process the data
print ("Reading data...")
d = list(parseData("./business.json"))
print ("Done!")

# Take a look at the output. Which features could be interesting to look at?
d[0]

### Part 1a: sklearn Introduction
We will use the `sklearn` library to do the heavy lifting in this section, already imported above. We will use the `stars` and `number of reviews` as the **features** (X, the variables that will predict) and a `business' open state` as the **label** (y, what to predict).

We first define `X` and `y` from our dataset. Recall that `X` is our matrix of features and `y` is the vector of labels corresponding to this matrix.

In [ ]:
def feature(datum):
    feat = [1, datum['stars'], datum['review_count']]
    return feat

In [ ]:
X = [feature(d) for d in data]
y = [d['is_open'] for d in data]

# Look at first 10 rows of X and y
print("Label: ", y[:10], "\nFeatures:", X[:10])

Now, let's use the `linear_model.LogisticRegression()` and `fit()` methods from the `sklearn` library to fit our model to our features and label. You can read the [sklearn documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) to find out more about the output.

In [ ]:
model = linear_model.LogisticRegression()
model.fit(X, y)

`predict` returns an array with the same number of elements as are in `X`. Each element will be a `1` or `0` -- these are the predictions for whether the corresponding business will be open. We will then see how many elements our model predicted correctly with a boolean statement.

In [ ]:
predictions = model.predict(X)
predictions

In [ ]:
correctPredictions = predictions == y
correctPredictions

Let's see how well our model performed. What is the percentage of predictions it got right?

In [ ]:
sum(correctPredictions) / len(correctPredictions)

### Part 1b. Logistic Regression with Training/Testing
We will follow the outline in Part 1a, but this time, we will perform train our model first and see how well it performs on data it hasn't seen before. In future weeks, we will learn how to validate our model.

First, let's load our libraries and read in the data like before. Note that we are using the `random` library to shuffle our dataset.

In [ ]:
import random
from sklearn import linear_model
import urllib.request # read and open URLs

def parseData(filename):
  for line in urllib.request.urlopen(filename):
    yield eval(line)

# Process the data
d = list(parseData("./business.json"))
print ("Done!")

Let's simplify our dataset to our previous features and label. Note that this time, we are keeping both the features and label together so we can correctly shuffle them without messing up the data. After shuffling the rows, we define and separate `X` and `y` again.

In [ ]:
# Simplify our dataset. 
def feature(datum):
    feat = [1, datum['stars'], datum['review_count'], datum['is_open']]
    return feat

dataset = [feature(d) for d in data]
dataset[0]

In [ ]:
random.shuffle(dataset)

In [ ]:
# Recall: arr[-1] in Python means the last index.
X = [values[:-1] for values in dataset]
y = [values[-1] for values in dataset]

We split our features and label into our training and test sets at a 50/50 ratio. You can try changing this ratio and see how your model changes.

In [ ]:
N = len(X)
X_train = X[:N//2]
X_test = X[N//2:]
y_train = y[:N//2]
y_test = y[N//2:]

len(X), len(X_train), len(X_test) # number of entries

Now we can train our model, again using `sklearn`'s logistic regression function. Note that we are passing in the training dataset we just defined.

In [ ]:
model = linear_model.LogisticRegression()
model.fit(X_train, y_train)

Now that our model is trained, let's see how well it performs on our test dataset and compare it to our training dataset.

In [ ]:
# Model's predictions
predictionsTrain = model.predict(X_train)
predictionsTest = model.predict(X_test)

# Whether model prediction was correct
correctPredictionsTrain = predictionsTrain == y_train
correctPredictionsTest = predictionsTest == y_test

In [ ]:
sum(correctPredictionsTrain) / len(correctPredictionsTrain) # Training accuracy

In [ ]:
sum(correctPredictionsTest) / len(correctPredictionsTest) # Test accuracy

Our model performed decently well! You can go back and run logistic regression using your own dataset or with different / more features. Experiment!

---

## Part 2: Nearest Neighbors

#### Documentation

* https://scikit-learn.org/stable/modules/neighbors.html#classification
* https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier

#### Introduction

*"The principle behind nearest neighbor methods is to find a predefined number of training samples closest in distance to the new point, and predict the label from these."* [-sklearn documentation](https://scikit-learn.org/stable/modules/neighbors.html)

To elaborate on this quote, imagine you have the following table, where $X_n$ are the coordinates in a three-dimensional space and $Y$ is the label. Suppose we use $X_1 = X_2 = X_3 = 0$ as our selected point using K-nearest neighbors. Using this dataset, we can make a prediction for $Y$ by:

1. Compute the (Euclidean) distance between each observation and our test point. [(Euclidean distance formula)](https://imgur.com/7z5rJcH.png)
2. Choose the K-nearest neighbors.
3. See what label the majority of the neighbors correspond to.
4. Label our test point with the majority label.

| Obs  | $X_1$ | $X_2$ | $X_3$ | Distance from $0,0,0$  |  $Y$ |
|------|------|------|------|------|
|   1  | 0 | 3 | 0| 3    | Red |
|   2  | 2 | 0 | 0| 2    | Red |
|   3  | 0 | 1 | 3|$\sqrt{10}$| Red |
|   4  | 0 | 1 | 2|$\sqrt{5}$| Green |
|   5  | -1 | 0 | 1|$\sqrt{2}$| Green |
|   6  | 1 | 1 | 1|$\sqrt{3}$| Red |

Take `K = 1`. Our test point at $0, 0, 0$ would be `Green`, since the 1 nearest neighbor is `Green`.

However, `K = 3` would result in our test point labeled as `Red`. Observations 2, 5, and 6 are the 3 closest neighbors and are `Red`, `Green`, `Red`, respectively. Since there are more `Red` points, our selected point will be `Red` as well.

This is why changing `K` will affect your predictions!

In [ ]:
# Same as above, except we import K-Nearest-Neighbors instead
import random
from sklearn.neighbors import KNeighborsClassifier 
import urllib.request # read and open URLs

def parseData(filename):
  for line in urllib.request.urlopen(filename):
    yield eval(line)

# Process the data
data = list(parseData("./business.json"))

# Simplify our dataset. 
def feature(datum):
    feat = [1, datum['stars'], datum['review_count'], datum['is_open']]
    return feat

dataset = [feature(d) for d in data]
dataset[0]

# Shuffle and define features/label
random.shuffle(dataset)
X = [values[:-1] for values in dataset]
y = [values[-1] for values in dataset]

# Split training/testing datasets
N = len(X)
X_train = X[:N//2]
X_test = X[N//2:]
y_train = y[:N//2]
y_test = y[N//2:]

print ("Done!")

**Now that we have our data set up, let's perform K-Nearest Neighbors!** Change the value of `n_neighbors` (a.k.a. `K`) and see how your predictions change. As `K` increases, it will be more accurate, but less flexible. Be careful about overfitting your data!

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(X_train, y_train) 

In [ ]:
# Model's predictions
predictionsTrain = neigh.predict(X_train)
predictionsTest = neigh.predict(X_test)

# Whether model prediction was correct
correctPredictionsTrain = predictionsTrain == y_train
correctPredictionsTest = predictionsTest == y_test

print("Training Accuracy: ", sum(correctPredictionsTrain) / len(correctPredictionsTrain)) # Training accuracy
print("Testing Accuracy: ", sum(correctPredictionsTest) / len(correctPredictionsTest)) # Test accuracy

---

## Part 3: Support Vector Machines

#### Documentation
* https://scikit-learn.org/stable/modules/svm.html#svm-classification
* https://scikit-learn.org/stable/modules/classes.html#module-sklearn.svm
* https://en.wikipedia.org/wiki/Support-vector_machine

#### Introduction

*"Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection."* [- sklearn documentation](https://scikit-learn.org/stable/modules/svm.html#svm)

Essentially, SVMs try to define some decision boundary (i.e. hyperplane) that distinctly classifies datapoints. To understand this a little better, imagine a 2D plane with two possible labels, such as below. 

![Basic hyperplane](https://imgur.com/kkfPVSK.png "Basic hyperplane")

There are many hyperplanes possible; SVMs try to maximize the distance from the training datapoints and the (non-linear) hyperplane. To enlarge the feature space and define a non-linear boundary, SVMs use **kernels**, functions that quantify the similarity of two observations. These can be polynomial or even radial! The explanation for how these work is a little too technical for this notebook, but we encourage you to read up on the inner workings of SVMs if you are interested.

![Non-linear boundaries](https://imgur.com/NQMDGTl.png "Non-linear boundaries")

#### Advantages of SVMs
* Effective in high dimensional spaces. Still effective in cases where number of dimensions is greater than the number of samples.
* Memory-efficient: uses a subset of training points in the decision function (called support vectors).
* Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

#### Disadvantages of SVMs
* If the number of features is much greater than the number of samples, avoiding over-fitting in choosing Kernel functions and regularization term is crucial.
* SVMs do not directly provide probability estimates. These are calculated using an expensive five-fold cross-validation.

In [1]:
# Same as above, except we import SVMs instead
import random
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import urllib.request # read and open URLs

def parseData(filename):
  for line in urllib.request.urlopen(filename):
    yield eval(line)

# Process the data
data = list(parseData("./business.json"))

# Simplify our dataset. 
def feature(datum):
    feat = [1, datum['stars'], datum['review_count'], datum['is_open']]
    return feat

dataset = [feature(d) for d in data]
dataset[0]

# Shuffle and define features/label
random.shuffle(dataset)
X = [values[:-1] for values in dataset]
y = [values[-1] for values in dataset]

# Split training/testing datasets
N = len(X)
X_train = X[:N//2]
X_test = X[N//2:]
y_train = y[:N//2]
y_test = y[N//2:]

print ("Done!")

Done!


In [3]:
clf = SVC(kernel='linear')
clf.fit(X_train,y_train)

# Predict and calculate accuracy of model
y_pred = clf.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.819


Try re-running the two cells above again. Do you get the same accuracy? Why or why not?

SVM Source: https://towardsdatascience.com/support-vector-machine-introduction-to-machine-learning-algorithms-934a444fca47

---

## You're all done!
You should be familiar with the basics of classification in Python by now. We encourage you to explore further by using your own datasets and thinking about research questions you can answer with classification. You will have a chance to show off your classification skills at the end of this course with your very own project!